In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


The training procedure follows 'Conformal Language Modeling'

In [42]:
%cd /content/drive/My Drive/Conformal-alignment

/content/drive/My Drive/Conformal-alignment
Archive.zip  p10/  p12/
Archive.zip  p10/  p11/  p12/


In [ ]:
!unzip mimic-cxr-resized-224.zip

In [3]:
!pip install datasets
!pip install nltk
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 5.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-non

In [4]:
from torchvision.io import ImageReadMode, read_image
from torchvision.transforms import CenterCrop, ConvertImageDtype, Normalize, Resize
from torchvision.transforms.functional import InterpolationMode
from datasets import load_dataset
import os

# I. Load model, dataset and pre-process functions

### 1. Load dataset from json files

In [12]:
# Load dataset
base = "./code/output/ap_pa_per_dicom_id"
dataset = load_dataset(
    "json",
    data_files={
        "train": os.path.join(base, "train.jsonl"),
        "dev": os.path.join(base, "dev.jsonl"),
        "validate": os.path.join(base, "validate.jsonl"),
        "test": os.path.join(base, "test.jsonl"),
    }
)
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating dev split: 0 examples [00:00, ? examples/s]

Generating validate split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['study_id', 'dicom_id', 'subject_id', 'split', 'report', 'PerformedProcedureStepDescription', 'ViewPosition', 'Rows', 'Columns', 'StudyDate', 'StudyTime', 'ProcedureCodeSequence_CodeMeaning', 'ViewCodeSequence_CodeMeaning', 'PatientOrientationCodeSequence_CodeMeaning', 'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion', 'Lung Opacity', 'No Finding', 'Pleural Effusion', 'Pleural Other', 'Pneumonia', 'Pneumothorax', 'Support Devices', 'image_path'],
        num_rows: 43361
    })
    dev: Dataset({
        features: ['study_id', 'dicom_id', 'subject_id', 'split', 'report', 'PerformedProcedureStepDescription', 'ViewPosition', 'Rows', 'Columns', 'StudyDate', 'StudyTime', 'ProcedureCodeSequence_CodeMeaning', 'ViewCodeSequence_CodeMeaning', 'PatientOrientationCodeSequence_CodeMeaning', 'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum', 'Fract

### 2. Load base model to be fine-tuned

In [5]:
from transformers import VisionEncoderDecoderModel
import torch

model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(
        "google/vit-base-patch16-224-in21k",
        "gpt2",
        cache_dir=None,
        torch_dtype=torch.float32
    )


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.crossattention.c_attn.bias', 'h.0.crossattention.c_attn.weight', 'h.0.crossattention.c_proj.bias', 'h.0.crossattention.c_proj.weight', 'h.0.crossattention.q_attn.bias', 'h.0.crossattention.q_attn.weight', 'h.0.ln_cross_attn.bias', 'h.0.ln_cross_attn.weight', 'h.1.crossattention.c_attn.bias', 'h.1.crossattention.c_attn.weight', 'h.1.crossattention.c_proj.bias', 'h.1.crossattention.c_proj.weight', 'h.1.crossattention.q_attn.bias', 'h.1.crossattention.q_attn.weight', 'h.1.ln_cross_attn.bias', 'h.1.ln_cross_attn.weight', 'h.10.crossattention.c_attn.bias', 'h.10.crossattention.c_attn.weight', 'h.10.crossattention.c_proj.bias', 'h.10.crossattention.c_proj.weight', 'h.10.crossattention.q_attn.bias', 'h.10.crossattention.q_attn.weight', 'h.10.ln_cross_attn.bias', 'h.10.ln_cross_attn.weight', 'h.11.crossattention.c_attn.bias', 'h.11.crossattention.c_attn.weight', 'h.11.crossat

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

### 3. Load tokenizer and define image processor

Report tokenizer

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(
        "gpt2",
        use_fast=True
    )
tokenizer.eos_token = tokenizer.decode(tokenizer.eos_token_id)
tokenizer.bos_token = tokenizer.decode(tokenizer.bos_token_id)
tokenizer.pad_token = tokenizer.eos_token


model.config.eos_token_id = tokenizer.eos_token_id
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.pad_token_id = tokenizer.pad_token_id

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Image processor

In [7]:
from transformers import AutoImageProcessor
# image processer
image_processor = AutoImageProcessor.from_pretrained(
        "google/vit-base-patch16-224-in21k"
    )

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

Pre-processing functions for reports

In [8]:
first_split = "FINDINGS AND IMPRESSION:"
second_split = "FINDINGS:"
third_split = "IMPRESSION:"
fourth_split = "FINDINGS"

def get_before_findings(t):
    """We want to get the earliest beginning of a findings paragraph"""
    if first_split in t:
        return t.split(first_split)[0] + first_split
    if second_split in t:
        return t.split(second_split)[0] + second_split
    if third_split in t:
        return t.split(third_split)[0] + third_split
    return t.split(fourth_split)[0] + fourth_split


FINDINGS_STARTS = [first_split, second_split, third_split, fourth_split]
FINAL_REPORT_START = " " * 33 + "FINAL REPORT"

In [9]:
# We use torchvision for faster image pre-processing. The transforms are implemented as nn.Module,
# so we jit it to be faster.
class Transform(torch.nn.Module):
    def __init__(self, image_size, mean, std):
        super().__init__()
        self.transforms = torch.nn.Sequential(
            Resize([image_size], interpolation=InterpolationMode.BICUBIC),
            CenterCrop(image_size),
            ConvertImageDtype(torch.float),
            Normalize(mean, std),
        )

    def forward(self, x) -> torch.Tensor:
        """`x` should be an instance of `PIL.Image.Image`"""
        with torch.no_grad():
            x = self.transforms(x)
        return x

Processing functions to be applied to datasets

In [10]:
# processing functions
def tokenize_reports(examples):
  reports = list(examples['report'])
  prompts = [get_before_findings(report) for report in reports]
  labels = tokenizer(reports, max_length=512, padding="max_length", truncation=True)
  examples["labels"] = labels.input_ids
  return examples


# image_size = image_processor.size['height']
# image_transformations = Transform(image_size, image_processor.image_mean, image_processor.image_std)
# image_transformations = torch.jit.script(image_transformations)

# def transform_images(examples):
#   images = [read_image(os.path.join('./mimic-cxr-resized-224/', image_file), mode=ImageReadMode.RGB) for image_file in examples['image_path']]
#   examples["pixel_values"] = [image_transformations(image) for image in images]
#   # print(image_transformations(images[0]))
#   return examples

# IMAGE_COLUMN = 'img_path'
# def transform_images(examples):
#     images = [read_image(os.path.join('./mimic-cxr-resized-224/', image_file), mode=ImageReadMode.RGB) for image_file in examples[IMAGE_COLUMN]]
#     examples['pixel_values'] = image_processor(images, return_tensors="pt").pixel_values
#     return examples

In [11]:
class Transform(torch.nn.Module):
    def __init__(self, image_size, mean, std):
        super().__init__()
        self.transforms = torch.nn.Sequential(
            Resize([image_size], interpolation=InterpolationMode.BICUBIC),
            CenterCrop(image_size),
            ConvertImageDtype(torch.float),
            Normalize(mean, std),
        )

    def forward(self, x) -> torch.Tensor:
        """`x` should be an instance of `PIL.Image.Image`"""
        with torch.no_grad():
            x = self.transforms(x)

        return x

image_size = image_processor.size['height']
image_transformations = Transform(image_size, image_processor.image_mean, image_processor.image_std)
image_transformations = torch.jit.script(image_transformations)

IMAGE_COLUMN = 'image_path'
def my_read_img(path):
  if os.path.exists(os.path.join('./mimic-cxr-resized-224/', path)):
    return image_transformations(read_image(os.path.join('./mimic-cxr-resized-224/', path), mode=ImageReadMode.RGB))
  else:
    return None
    # return Image.open(os.path.join('./mimic-cxr-resized-224/', path)).convert('RGB')

def transform_images(examples):
    # image = read_image(os.path.join('./mimic-cxr-resized-224/', examples[IMAGE_COLUMN]), mode=ImageReadMode.RGB)
    # print(images)
    examples["pixel_values"] = [my_read_img(image_file) for image_file in examples['image_path']]
    print("succeeded!" + str(len(examples)))
    # examples["pixel_values"] = [image_transformations(image) for image in images]
    # examples["pixel_values"] = [image_transformations(read_image(os.path.join('./mimic-cxr-resized-224/', path)).convert('RGB')) for path in examples['image_path'] if os.path.exists(os.path.join('./mimic-cxr-resized-224/', path))]
    return examples

##### Some useless code for document

In [ ]:
# from torchvision import transforms
# test_transforms = transforms.Compose(
#     [
#         transforms.Resize(size=image_processor.size['height']),
#         transforms.CenterCrop(size=[
#             image_processor.size['height'],
#             image_processor.size['height'],
#         ]
#         ),
#         transforms.ToTensor(),
#         transforms.Normalize(
#             mean=image_processor.image_mean,
#             std=image_processor.image_std,
#         ),
#     ]
# )

# # def my_read_img(path):
# #   if os.path.exists(os.path.join('./mimic-cxr-resized-224/', path)):
# #     return Image.open(os.path.join('./mimic-cxr-resized-224/', path)).convert('RGB')


# from PIL import Image
# def my_transform_images(examples):
#   # examples["pixel_values"] = Image.open(os.path.join('./mimic-cxr-resized-224/', examples['image_path'])).convert('RGB')
#   examples["pixel_values"] = [test_transforms(Image.open(os.path.join('./mimic-cxr-resized-224/', path)).convert('RGB')) for path in examples['image_path'] if os.path.exists(os.path.join('./mimic-cxr-resized-224/', path))]
#   return examples

# II. Prepare datasets

## 1. Tokenize and transform images

In [ ]:
train_dataset = dataset["train"]
train_dataset = train_dataset.map(
            function=tokenize_reports,
            batched=True,
            num_proc=1,
            load_from_cache_file=False,
            desc="Running tokenizer on train dataset",
        )

train_dataset = train_dataset.map(
            function=transform_images,
            batched=True,
            num_proc=1,
            load_from_cache_file=False,
            desc="Running tokenizer on training dataset",
        )

Running tokenizer on training dataset:   0%|          | 0/9020 [00:00<?, ? examples/s]

succeeded!31
succeeded!31
succeeded!31
succeeded!31
succeeded!31
succeeded!31
succeeded!31
succeeded!31
succeeded!31
succeeded!31


In [ ]:
eval_dataset = dataset["validate"]
eval_dataset = eval_dataset.map(
            function=tokenize_reports,
            batched=True,
            num_proc=1,
            load_from_cache_file=False,
            desc="Running tokenizer on validation dataset",
        )
eval_dataset = eval_dataset.map(
            function=transform_images,
            batched=True,
            num_proc=1,
            load_from_cache_file=False,
            desc="Running tokenizer on validation dataset",
        )

## 2. Delete None entries where image does not exist

In [ ]:
def filter_none_entries(example):
    return example['pixel_values'] is not None

filtered_train_dataset = train_dataset.filter(filter_none_entries)
filtered_eval_dataset = eval_dataset.filter(filter_none_entries)

## 3. Select labels and pixel_values for training

In [ ]:
def select_columns(example):
    return {
        'labels': example['labels'],
        'pixel_values': example['pixel_values']
    }

ds_train = filtered_train_dataset.map(
    select_columns,
    remove_columns=[col for col in filtered_train_dataset.column_names if col not in ['labels', 'pixel_values']])
ds_eval = filtered_eval_dataset.map(
    select_columns,
    remove_columns = [col for col in filtered_train_dataset.column_names if col not in ['labels', 'pixel_values']]
)

#### Optional: save the processed datasets

(Seems CPU is faster in processing than GPU!)

In [ ]:
train_dataset.save_to_disk("./clm_processed_train/")
eval_dataset.save_to_disk("./clm_processed_validate/")

Saving the dataset (0/11 shards):   0%|          | 0/9020 [00:00<?, ? examples/s]

In [ ]:
filtered_train_dataset.save_to_disk("./processed_data/clm_processed_train/")
filtered_eval_dataset.save_to_disk("./clm_processed_validate/")

Saving the dataset (0/11 shards):   0%|          | 0/9015 [00:00<?, ? examples/s]

Saving the dataset (0/3 shards):   0%|          | 0/1876 [00:00<?, ? examples/s]

In [ ]:
ds_train.save_to_disk("./processed_data/clm_processed_minimal_train/")
ds_eval.save_to_disk("./processed_data/clm_processed_minimal_validate/")

Saving the dataset (0/11 shards):   0%|          | 0/9015 [00:00<?, ? examples/s]

Saving the dataset (0/3 shards):   0%|          | 0/1876 [00:00<?, ? examples/s]

# III. Model training following clm

In [ ]:
from transformers.trainer import get_last_checkpoint
from transformers.trainer import Trainer
from transformers import TrainingArguments
from transformers import default_data_collator

training_args = TrainingArguments(
    evaluation_strategy="steps",
    eval_steps = 500,
    num_train_epochs = 10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    output_dir="./saved_checkpoints/clm-cxr-report-generation-output",
    save_strategy='steps',
    save_steps=1000
)

trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=filtered_train_dataset,
        eval_dataset=filtered_eval_dataset,
        data_collator=default_data_collator,
    )

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
checkpoint = None

train_result = trainer.train(resume_from_checkpoint=checkpoint)
trainer.save_model()
trainer.log_metrics("train", train_result.metrics)
trainer.save_metrics("train", train_result.metrics)
trainer.save_state()

Step,Training Loss,Validation Loss
500,0.617300,0.474830
1000,0.468600,0.431589
1500,0.431700,0.412789
2000,0.407100,0.397710
2500,0.381800,0.389804
3000,0.372600,0.381234
3500,0.360000,0.381270
4000,0.351800,0.371579
4500,0.349800,0.368561
5000,0.330100,0.368844


Removed shared tensor {'decoder.lm_head.weight'} while saving. This should be OK, but check by verifying that you don't receive any warning while reloading


***** train metrics *****
  epoch                    =          10.0
  total_flos               = 15151519127GF
  train_loss               =        0.3486
  train_runtime            =    4:57:21.10
  train_samples_per_second =         5.053
  train_steps_per_second   =         0.632
